In [1]:
import os
import pandas as pd
import chardet
from sqlalchemy import create_engine

In [2]:
# Set working directory
os.chdir(r"C:\Users\kche8\Documents\Data Project")
print(os.getcwd())  

C:\Users\kche8\Documents\Data Project


In [3]:
# Detect file encoding
with open('Walmart.csv', 'rb') as f:
    result = chardet.detect(f.read(100000))

encoding = result['encoding']
print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [4]:
# Load dataset
df = pd.read_csv(r"C:\Users\kche8\Desktop\Dataset\Walmart.csv", encoding=encoding)
df.shape

(10051, 11)

In [5]:
df.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,$74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,$15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,$46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,$58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,$86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


In [6]:
df.describe()

,invoice_id,quantity,rating,profit_margin
count,10051.000000,10020.000000,10051.000000,10051.000000
mean,5025.741220,2.353493,5.825659,0.393791
std,2901.174372,1.602658,1.763991,0.090669
min,1.000000,1.000000,3.000000,0.180000
25%,2513.500000,1.000000,4.000000,0.330000
50%,5026.000000,2.000000,6.000000,0.330000
75%,7538.500000,3.000000,7.000000,0.480000
max,10000.000000,10.000000,10.000000,0.570000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      10051 non-null  int64  
 1   Branch          10051 non-null  object 
 2   City            10051 non-null  object 
 3   category        10051 non-null  object 
 4   unit_price      10020 non-null  object 
 5   quantity        10020 non-null  float64
 6   date            10051 non-null  object 
 7   time            10051 non-null  object 
 8   payment_method  10051 non-null  object 
 9   rating          10051 non-null  float64
 10  profit_margin   10051 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 863.9+ KB


In [8]:
df.duplicated().sum()

51

In [9]:
df.isnull().sum()

invoice_id         0
Branch             0
City               0
category           0
unit_price        31
quantity          31
date               0
time               0
payment_method     0
rating             0
profit_margin      0
dtype: int64

In [10]:
# Remove duplicates
duplicates_count = df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.shape

(10000, 11)

In [11]:
# Drop rows with nulls in critical fields
missing_counts = df.isnull().sum()
df.dropna(subset=['unit_price', 'quantity'], inplace=True)
df.shape

(9969, 11)

In [12]:
# Clean unit_price and convert to float
df['unit_price'] = (df['unit_price']
                    .str.replace('$', '', regex=False)
                    .str.replace(',', '', regex=False)
                    .astype(float))

In [13]:
# Standardize column names
df.columns = df.columns.str.lower().str.strip()

In [14]:
# Create total
df['total'] = df['unit_price'] * df['quantity']

In [15]:
# Convert date to consistent format
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y').dt.strftime('%m/%d/%y')

In [16]:
# Check unique categorical values
df['unit_price'].min() - df['unit_price'].max()

-89.88

In [17]:
df['quantity'].min(), - df['quantity'].max()

(1.0, -10.0)

In [18]:
df['total'].min(), df['total'].max()

(10.17, 993.0)

In [19]:
# Check unique categorical values
df['payment_method'].unique()

array(['Ewallet', 'Cash', 'Credit card'], dtype=object)

In [20]:
df['branch'].unique()

array(['WALM003', 'WALM048', 'WALM067', 'WALM064', 'WALM013', 'WALM026',
       'WALM088', 'WALM100', 'WALM066', 'WALM065', 'WALM035', 'WALM027',
       'WALM031', 'WALM008', 'WALM029', 'WALM061', 'WALM053', 'WALM044',
       'WALM083', 'WALM023', 'WALM001', 'WALM072', 'WALM046', 'WALM012',
       'WALM075', 'WALM076', 'WALM098', 'WALM021', 'WALM096', 'WALM022',
       'WALM095', 'WALM041', 'WALM070', 'WALM030', 'WALM033', 'WALM079',
       'WALM042', 'WALM036', 'WALM056', 'WALM058', 'WALM015', 'WALM063',
       'WALM089', 'WALM093', 'WALM045', 'WALM005', 'WALM078', 'WALM037',
       'WALM069', 'WALM087', 'WALM020', 'WALM007', 'WALM017', 'WALM094',
       'WALM071', 'WALM090', 'WALM097', 'WALM043', 'WALM011', 'WALM049',
       'WALM040', 'WALM034', 'WALM014', 'WALM004', 'WALM057', 'WALM055',
       'WALM038', 'WALM025', 'WALM024', 'WALM085', 'WALM016', 'WALM018',
       'WALM068', 'WALM002', 'WALM062', 'WALM052', 'WALM047', 'WALM054',
       'WALM099', 'WALM074', 'WALM028', 'WALM019', 

In [21]:
df['category'].unique()

array(['Health and beauty', 'Electronic accessories',
       'Home and lifestyle', 'Sports and travel', 'Food and beverages',
       'Fashion accessories'], dtype=object)